In [1]:
import pandas as pd

# Import OHLC + indicators
ohlc = pd.read_csv("applyiedMacd-1.csv", parse_dates=["DateTime"])
ohlc.set_index("DateTime", inplace=True)

# Import trades
trades = pd.read_csv("bt.result.csv", parse_dates=["Entry Time", "Exit Time"])

# Take first 100 rows
ohlc_100 = ohlc.iloc[:500]


In [ ]:



import matplotlib.patches as patches
import mplfinance as mpf

# Plot all available OHLC data
ohlc_all = ohlc.copy()
total_candles = len(ohlc_all)

# Function to create rectangles for all trades
def get_rectangles(trades_df):
    rects = []
    for _, trade in trades_df.iterrows():
        entry_idx = max(0, trade['Candle Index Entry'])
        exit_idx = max(entry_idx, trade['Candle Index Exit'])  # ensure exit >= entry

        # Skip if indices are outside OHLC range
        if entry_idx >= total_candles:
            continue
        exit_idx = min(exit_idx, total_candles-1)

        # SL rectangle
        rects.append(
            patches.Rectangle(
                (entry_idx - 0.5, min(trade['SL'], trade['Entry Price'])),
                width=exit_idx-entry_idx+1,
                height=abs(trade['Entry Price']-trade['SL']),
                color='red', alpha=0.3
            )
        )
        # TP rectangle
        rects.append(
            patches.Rectangle(
                (entry_idx - 0.5, min(trade['TP'], trade['Entry Price'])),
                width=exit_idx-entry_idx+1,
                height=abs(trade['TP']-trade['Entry Price']),
                color='green', alpha=0.3
            )
        )
    return rects

rectangles = get_rectangles(trades)

# Plot chart full-width
fig, ax = mpf.plot(
    ohlc_all,
    type='candle',
    style='charles',
    returnfig=True,
    volume=True,
    figsize=(25,8),
    title='Candlestick with SL/TP for All Trades'
)

# Add all rectangles
for rect in rectangles:
    ax[0].add_patch(rect)

fig.show()
